### 방법 1. from_template() 메소드를 사용하여 PromptTemplate 객체 생성
* 아주 간단하게 프롬트 생성
* 여러 변수도 처리가능

In [2]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3.1", temperature=0, max_length=135)

template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country"],
)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [3]:
# format은 템플릿 문자열에서 지정된 변수 대체, 실제 값을 대입하는건 아니고, 확인만 하는용도
# 체인의 과정에서 format은 필요없다. 실제 삽입은 invoke를 통해 한다. 
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

### 방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성
* PromptTemplate 객체를 생성할 때 input_variables를 명시적으로 지정하는게 첫번째와 다르다.
* 더 나은점은 input_variables를 명시적으로 지정하여 템플릿 문자열과 일치하지 않는 경우 예외를 발생시켜. 코드의 안정성을 높이고, 더 복잡한 템플릿을 다룰 수 있다.

In [4]:
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    # 일부 변수만 선언후 넣어놈 
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

prompt

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [5]:
# 나머지 변수 삽입
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [6]:
# 새로운 prompt_partial 만들고 캐나다 일부변수로 삽입
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [7]:
# 체인 생성
chain =prompt_partial | llm

# invoke 메소드는 LangChain 라이브러리에서 체인을 실행하는 데 사용. 
# 이 메소드는 프롬프트 템플릿에 입력 변수를 전달하고, 이를 기반으로 모델이 응답을 생성하도록 함
chain.invoke("대한민국")

AIMessage(content='대한민국의 수도는 서울입니다. 캐나다의 수도는 오타와입니다.', response_metadata={'model': 'llama3.1', 'created_at': '2024-08-07T05:00:26.6307439Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 8247940100, 'load_duration': 4405396900, 'prompt_eval_count': 24, 'prompt_eval_duration': 1231724000, 'eval_count': 22, 'eval_duration': 2609071000}, id='run-ffb31d58-85da-4643-ae8b-4c685790b7c9-0')

In [8]:
# content로 결과물로부터 내용만 추출
chain.invoke("대한민국").content

'대한민국의 수도는 서울입니다. 캐나다의 수도는 오타와입니다.'

In [9]:
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

'대한민국의 수도는 서울입니다. 호주의 수도는 캔버라입니다.'

In [10]:
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["country1","country2"] # 이번엔 두 변수다 입력 받게 해봄
)

prompt

PromptTemplate(input_variables=['country1', 'country2'], template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [11]:
# 입력값을 변수가 아닌 함수로 줘봄

from datetime import datetime

# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # dictionary 형태로 partial_variables를 전달
    },
)

# format을 통해 변수에 n=3을 넣어보고 양식 확인. 
prompt.format(n=3)

'오늘의 날짜는 August 07 입니다. 오늘이 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [12]:
# chain 을 생성.
chain = prompt | llm

In [13]:
# chain에 실제로 n=3 을 삽입 후 결과를 확인합니다.
chain.invoke(3).content

'생일이 August 7 인 유명인 3 명은 다음과 같습니다.\n\n1. 제시카 비엘 (Jessica Biel, 1982년 3월 3 일 ~ ) - 미국의 배우\n2. 마이클 J. 폭스 (Michael J. Fox, 1961년 6월 9 일 ~ ) - 캐나다의 배우\n3. 제니퍼 로렌스 (Jennifer Lawrence, 1990년 8월 15 일 ~ ) - 미국의 배우'

In [14]:
# 오늘 날짜가 아닌 다른 날짜도 삽입해봄
chain.invoke({"today": "Jan 02", "n": 3}).content

'1. 제니퍼 로페즈(Jennifer Lopez, 1969년 7월 24일)\n2. 마이클 J. 폭스(Michael J. Fox, 1961년 6월 9일)\n3. 조지 해리슨(George Harrison, 1943년 2월 25일)'

### 파일로부터 template 읽어오기
* yaml 파일에 template 구조를 저장하고 불러옴

```
[yaml 파일 내용]
template: "{fruit}의 색깔이 뭐야?"
input_variables:
  - fruit
```

* 왜 YAML 파일을 사용하는가?
  * 가독성: YAML은 사람이 읽기 쉽고 이해하기 쉬운 형식으로, 데이터 구조가 복잡해질수록 유리.
  * 주석: YAML 파일에 주석을 추가할 수 있어, 데이터를 설명하거나 문서화하는 데 유용.
  * 구조 표현: 계층적 데이터를 직관적으로 표현할 수 있으며, 특히 설정 파일이나 구성 파일에서 자주 사용됩니다.

* 잘못된 방법 : load_prompt("prompts/fruit_color.yaml", encoding="cp949")

* LangChain의 load_prompt 함수는 기본적으로 인코딩을 지정할 수 있는 매개변수를 제공하지 않습니다. 따라서, 파일을 읽을 때 인코딩을 직접 지정할 수 있는 사용자 정의 함수 load_prompt_with_encoding를 작성

In [15]:
from langchain_core.prompts import load_prompt

import yaml
from langchain_core.prompts import PromptTemplate

def load_prompt_with_encoding(file_path, encoding='utf-8'):
    from pathlib import Path
    file_path = Path(file_path)
    if file_path.suffix.endswith((".yaml", ".yml")):
        with open(file_path, "r", encoding=encoding) as f:
            config = yaml.safe_load(f)
    else:
        raise ValueError(f"Got unsupported file type {file_path.suffix}")
    
    # PromptTemplate 생성
    return PromptTemplate(**config)

# 예시 사용
try:
    prompt = load_prompt_with_encoding("study.yaml", encoding='utf-8')
except UnicodeDecodeError:
    prompt = load_prompt_with_encoding("study.yaml", encoding='cp949')

prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [16]:
try:
    prompt2 = load_prompt_with_encoding("study2.yaml", encoding='utf-8')
except UnicodeDecodeError:
    prompt2 = load_prompt_with_encoding("study2.yaml", encoding='cp949')

# 변수를 임시로 삽입해보고 실제로 삽인된건지 확인
prompt2.format(country="대한민국")
chain = prompt2 | llm
chain
# 결과를 보면 삽입 안되있는걸 알수 있음. 삽입은 invoke로

PromptTemplate(input_variables=['country'], template='{country}의 수도에 대해서 알려주세요.\n수도의 특징을 다음의 양식에 맞게 정리해 주세요.\n300자 내외로 작성해 주세요.\n한글로 작성해 주세요.\n----\n[양식]\n1. 면적\n2. 인구\n3. 역사적 장소\n4. 특산품\n\n#Answer:\n')
| ChatOllama(model='llama3.1', temperature=0.0)

In [17]:
result = chain.invoke({"country": "대한민국"})
result.content

'대한민국의 수도는 서울입니다.\n\n**서울의 특징**\n\n1. **면적**: 605.25km² (국토의 약 0.6%를 차지)\n2. **인구**: 약 10,219,000명 (2020년 기준)\n3. **역사적 장소**: 고대 한양시(서울)의 역사를 지닌 곳으로, 삼국시대부터 조선시대까지의 역사와 문화가 살아남은 곳\n4. **특산품**: 서울에는 다양한 특산품이 있습니다. 대표적인 것들은 다음과 같습니다.\n * **한강수**: 한강에서 채집한 수로, 한국의 대표적인 생수입니다.\n * **서울식당**: 다양한 종류의 음식을 제공하는 식당으로, 한국의 대표적인 음식문화를 느낄 수 있는 곳입니다.\n * **명동시장**: 역사와 문화가 살아남은 시장으로, 다양한 물건을 판매하는 곳입니다.'

### ChatPromptTemplate
* 대화목록을 프롬프트로 주입하고자 할 때 활용
* 메시지는 튜플(tuple) 형식으로 구성하며, (role, message) 로 구성하여 리스트로 생성
* role ["system": 시스템 설정 메시지. 주로 전역설정과 관련된 프롬프트. - "human" : 사용자 입력 메시지. - "ai": AI 의 답변 메시지]

In [18]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [19]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 변수를 넣어서 확인해봄
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디 입니다.'),
 HumanMessage(content='반가워요!'),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?'),
 HumanMessage(content='당신의 이름은 무엇입니까?')]

In [20]:
llm.invoke(messages).content

'내 이름은 테디(Teddy)입니다! 반갑게 만나요!'

In [21]:
chain = chat_template | llm
chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"}).content

'내 이름은 테디(Teddy)야! 반가워! 어떻게 할 수 있는지 물어보면 좋겠어!'

### MessagePlaceholder
* 프롬프트 템플릿에서 변수를 정의하고, 실제 값을 대입하는 데 사용
* 사용처
    1. 대화형 AI 응답 생성: 대화형 AI 모델이 사용자 입력에 따라 다양한 응답을 생성해야 할 때, MessagePlaceholder를 사용하여 프롬프트 템플릿을 동적으로 생성할 수 있습니다.
    2. 자동화된 이메일 또는 보고서 생성: 다양한 변수를 포함하는 자동화된 이메일, 보고서, 알림 등을 생성할 때 유용합니다. 예를 들어, 사용자 이름, 날짜, 특정 데이터 포인트 등을 동적으로 대입하여 문서를 생성할 수 있습니다.
    3. 데이터 처리 및 분석: 데이터 처리 파이프라인에서 변수를 사용하여 다양한 데이터 포인트를 대입하고, 이를 기반으로 분석 결과를 생성할 수 있습니다.

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# from_messages는 LangChain 라이브러리에서 프롬프트 템플릿을 여러 메시지를 결합하여 생성하는 데 사용
chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        # MessagesPlaceholder는 실제 대화 내용을 대입할 자리 표시자 역할
        # variable_name으로 conversation을 지정하여, 실제 대화가 이 위치에 삽입
        MessagesPlaceholder(variable_name="conversation"),
        # 인간 사용자 메시지로, AI에게 대화를 특정 단어 수로 요약하도록 지시
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.')), MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='지금까지의 대화를 {word_count} 단어로 요약합니다.'))])

In [23]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

formatted_chat_prompt

'System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.\nHuman: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.\nAI: 반가워요! 앞으로 잘 부탁 드립니다.\nHuman: 지금까지의 대화를 5 단어로 요약합니다.'

In [24]:
# chain 생성
chain = chat_prompt | llm
chain

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.')), MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='지금까지의 대화를 {word_count} 단어로 요약합니다.'))])
| ChatOllama(model='llama3.1', temperature=0.0)

In [25]:
# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
).content

'테디와 인사하기'

In [26]:
# StrOutputParser()을 추가하고 위의 체인과 비교
chain = chat_prompt | llm | StrOutputParser()

# content를 안써도 똑같은 결과가 나옴
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

'테디와 인사하기'

### 퓨샷 프롬프트(FewShotPromptTemplate)
* 모델이 수행해야 하는 작업이 복잡하거나 구조화된 경우, 몇 가지 예제를 제공함으로써 모델이 그 패턴을 학습하고 더 정확하게 응답할 수 있도록 도움을 줌
* 일반적인 NLP 태스크(번역, 텍스트 생성 등)에서 Few-Shot Learning을 적용하기 위해 사용

In [27]:
# 결과를 실시간 스트리밍 출력 하는법, langchain_teddynote 모듈 설치
from langchain_teddynote.messages import stream_response

# 질의내용
question = "문재인에 대해 알려줘"

# 질의
answer = llm.stream(question)
stream_response(answer)

문재인은 2017년부터 2022년까지 대한민국의 제19대 대통령을 지낸 정치인이다. 그는 민주평통, 새정치국민회의, 더불어민주당에서 활동한 경력을 가지고 있으며, 2017년 대선에서 승리하여 대통령직에 취임하였다.

문재인은 1953년 12월 24일 전라남도 해남군에서 태어났다. 그는 1975년에 서울대학교 법과대학을 졸업하고, 1980년대 초반부터 정치 활동을 시작했다. 그는 민주당의 당원으로 활동했으며, 1991년 국회 의원 선거에 출마하여 당선되었다.

문재인은 2004년 새정치국민회의 창당에 참여하였고, 2007년 대선에서 문희상과 함께 공동대표를 맡았다. 그는 2012년 대선에서 안철수와의 경쟁을 벌였지만, 패배하였다. 그러나 2017년 대선에서는 승리하여 대통령직에 취임하였다.

문재인은 대통령으로 재직할 때, 경제 성장과 사회 공정성, 교육 및 연구 개발, 환경 보호 등 다양한 정책들을 추진했다. 그는 또한 북한과의 관계 개선과 평화적 해결을 위해 노력했으며, 2018년 평창 동계 올림픽의 성공적인 개최를 위한 노력을 기울였다.

문재인은 2022년 5월 9일에 대통령직에서 물러나고, 더불어민주당 대표로 재직 중이다.

In [28]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

example_prompt.format(**examples[0])

'Question:\n스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?\nAnswer:\n이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 스티브 잡스는 몇 살에 사망했나요?\n중간 답변: 스티브 잡스는 56세에 사망했습니다.\n추가 질문: 아인슈타인은 몇 살에 사망했나요?\n중간 답변: 아인슈타인은 76세에 사망했습니다.\n최종 답변은: 아인슈타인\n'

In [29]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
final_prompt

'Question:\n스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?\nAnswer:\n이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 스티브 잡스는 몇 살에 사망했나요?\n중간 답변: 스티브 잡스는 56세에 사망했습니다.\n추가 질문: 아인슈타인은 몇 살에 사망했나요?\n중간 답변: 아인슈타인은 76세에 사망했습니다.\n최종 답변은: 아인슈타인\n\n\nQuestion:\n네이버의 창립자는 언제 태어났나요?\nAnswer:\n이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 네이버의 창립자는 누구인가요?\n중간 답변: 네이버는 이해진에 의해 창립되었습니다.\n추가 질문: 이해진은 언제 태어났나요?\n중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.\n최종 답변은: 1967년 6월 22일\n\n\nQuestion:\n율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?\nAnswer:\n이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 율곡 이이의 어머니는 누구인가요?\n중간 답변: 율곡 이이의 어머니는 신사임당입니다.\n추가 질문: 신사임당은 언제 태어났나요?\n중간 답변: 신사임당은 1504년에 태어났습니다.\n추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?\n중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.\n최종 답변은: 연산군\n\n\nQuestion:\n올드보이와 기생충의 감독이 같은 나라 출신인가요?\nAnswer:\n이 질문에 추가 질문이 필요한가요: 예.\n추가 질문: 올드보이의 감독은 누구인가요?\n중간 답변: 올드보이의 감독은 박찬욱입니다.\n추가 질문: 박찬욱은 어느 나라 출신인가요?\n중간 답변: 박찬욱은 대한민국 출신입니다.\n추가 질문: 기생충의 감독은 누구인가요?\n중간 답변: 기생충의 감독은 봉준호입니다.\n추가 질문: 봉준호는 어느 나라 출신인가요?\n중간 답변: 봉준호는 대한민국 출신입니다.\n최종 답변은: 예\n\n\nQuestion:\nGoogle이 창립된 연도에 

In [30]:
# 결과 출력
answer = llm.stream(final_prompt)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google이 창립된 연도는 언제인가요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates의 나이는 몇 살인가요? (1998년)
중간 답변: Bill Gates는 1998년 10월에 태어났습니다. 따라서, 1998년이 되면 0세입니다.
최종 답변은: 0

In [31]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# chain 생성
chain = prompt | llm | StrOutputParser()

# 결과 출력
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

이 질문에 추가 질문이 필요한가요: 예.
추가 질문: Google이 창립된 연도는 언제인가요?
중간 답변: Google은 1998년에 창립되었습니다.
추가 질문: Bill Gates의 나이는 몇 살인가요? (1998년)
중간 답변: Bill Gates는 1998년 10월에 태어났습니다. 따라서, 1998년이 되면 0살입니다.
최종 답변은: 0

### Example Selector
* 예제가 많은 경우 프롬프트에 포함할 예제를 선택해야 할 수도 있다.
* 여러 유형의 예시 중 유사성이 높은 예시를 선택

In [32]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import CacheBackedEmbeddings

embeddings = HuggingFaceEmbeddings(
            model_name='jhgan/ko-sroberta-nli',
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': True},
        )
vectorstore = Chroma.from_texts("example_selector", embeddings)

# Vector DB 생성 (저장소 이름, 임베딩 클래스)
chroma = Chroma("example_selector", embeddings)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    embeddings,
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    vectorstore,
    # 이것은 생성할 예시의 수입니다.
    k=1,
)

# 입력과 가장 유사한 예시를 선택합니다.
selected_examples = example_selector.select_examples({"question": question})

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

# 입력에 가까운 예시의 질문을 추출
print(f"입력에 가장 유사한 예시:\n{question}\n")

# 그 질문의 내용들을 출력
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

c:\Users\weare\pro-ollama\pdf-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


입력에 가장 유사한 예시:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?

question:
네이버의 창립자는 언제 태어났나요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일



In [33]:
# 예시를 선정했으니, 질문을 삽입해서 프롬트를 전부 출력해봄

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
example_selector_prompt = prompt.format(question=question)
print(example_selector_prompt)

Question:
네이버의 창립자는 언제 태어났나요?
Answer:
이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일


Question:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?
Answer:


In [34]:
# 체인 생성
chain = prompt | llm
chain

FewShotPromptTemplate(input_variables=['question'], example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000029FCAC7F010>, k=1, example_keys=None, input_keys=None, vectorstore_kwargs=None), example_prompt=PromptTemplate(input_variables=['answer', 'question'], template='Question:\n{question}\nAnswer:\n{answer}'), suffix='Question:\n{question}\nAnswer:')
| ChatOllama(model='llama3.1', temperature=0.0)

In [35]:
# 이제 체인을 다 만들었으니 결과를 출력 해보자
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

Bill Gates가 Google을 창립한 것은 아닙니다. Google의 창립자는 Larry Page와 Sergey Brin입니다.

Bill Gates는 Microsoft를 창립했습니다.

Bill Gates는 1955년 10월 28일에 태어났습니다.

Google이 창립된 연도는 1998년입니다.

1998년 Bill Gates의 나이는 43살입니다.

### FewShotChatMessagePromptTemplate
* FewShotPromptTemplate는 일반적인 NLP 태스크(번역, 텍스트 생성 등)에서 Few-Shot Learning을 적용하기 위해 사용
* FewShotChatMessagePromptTemplate는 대화형 AI 모델(챗봇 등)에서 Few-Shot Learning을 적용하기 위해 사용

In [41]:
examples = [
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
        회의록: XYZ 회사 마케팅 전략 회의
        일시: 2023년 12월 25일
        장소: XYZ 회사 회의실
        참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

        1. 개회
        - 회의는 김수진 팀장의 개회사로 시작됨.
        - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

        2. 시장 동향 개요 (김수진)
        - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
        - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

        3. 디지털 마케팅 전략 (박지민)
        - 박지민은 디지털 마케팅 전략에 대해 발표.
        - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

        4. 소셜 미디어 캠페인 (이준호)
        - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
        - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

        5. 종합 논의
        - 팀원들 간의 아이디어 공유 및 토론.
        - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

        6. 마무리
        - 다음 회의 날짜 및 시간 확정.
        - 회의록 정리 및 배포는 박지민 담당.
        """,
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]

from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
)
from langchain_community.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(
            model_name='jhgan/ko-sroberta-nli',
            model_kwargs={'device': 'cuda'},
            encode_kwargs={'normalize_embeddings': True},
        )
vectorstore = Chroma.from_texts("example_selector", embeddings)

# Vector DB 생성 (저장소 이름, 임베딩 클래스)
chroma = Chroma("example_selector", embeddings)



example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    embeddings,
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    vectorstore,
    # 이것은 생성할 예시의 수입니다.
    k=1,
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
)

In [42]:
# 질문 작성
question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}

# 질문과 유사한 예제 선택
example_selector.select_examples(question)

[{'answer': '\n회의록: XYZ 회사 마케팅 전략 회의\n일시: 2023년 12월 25일\n장소: XYZ 회사 회의실\n참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)\n\n1. 개회\n   - 회의는 김수진 팀장의 개회사로 시작됨.\n   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.\n\n2. 시장 동향 개요 (김수진)\n   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.\n   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.\n\n3. 디지털 마케팅 전략 (박지민)\n   - 박지민은 디지털 마케팅 전략에 대해 발표.\n   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.\n\n4. 소셜 미디어 캠페인 (이준호)\n   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.\n   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.\n\n5. 종합 논의\n   - 팀원들 간의 아이디어 공유 및 토론.\n   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.\n\n6. 마무리\n   - 다음 회의 날짜 및 시간 확정.\n   - 회의록 정리 및 배포는 박지민 담당.\n',
  'input': '2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.',
  'instruction': '당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요'}]

In [43]:

final_prompt = ChatPromptTemplate.from_messages(
    [
        # 모델에게 주어진 역할이나 태스크를 명시적으로 지시
        (
            "system",
            "You are a helpful assistant.",
        ),
        # 적절한 예시 선택기
        few_shot_prompt,
        # 대화의 주체가 인간 사용자인 경우를 나타냅니다. 이는 모델이 응답을 생성할 때 해당 발화가 
        # 사용자로부터 온 것임을 명확히 알 수 있게 함. "ai" 또는 "assistant"라면 모델의 응답을 나타냄. 
        # 이것으로 인해 모델이 생성한 응답임을 명확히 구분할수 있다.
        ("human", "{instruction}\n{input}"),
    ]
)

# chain 생성
chain = final_prompt | llm

# 실행 및 결과 출력
answer = chain.stream(question)
stream_response(answer)

회의록: ABC 기술 회사의 제품 개발 팀 주간 진행 상황 회의
일시: 2023년 12월 26일
장소: ABC 기술 회사 회의실
참석자: 최현수 (프로젝트 매니저), 황지연 (주요 개발자), 김태영 (UI/UX 디자이너)

1. 개회
   - 회의는 최현수 프로젝트 매니저의 개회사로 시작됨.
   - 회의의 목적은 새로운 모바일 애플리케이션 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획 수립.

2. 프로젝트 진행 상황 (최현수)
   - 최현수는 프로젝트의 현재 진행 상황에 대해 보고.
   - 개발 단계, 테스트 단계 및 출시 준비에 대한 업데이트를 공유.

3. 개발 업데이트 (황지연)
   - 황지연은 개발 작업에 대한 업데이트를 제공.
   - 주요 기능 구현과 버그 수정에 대한 진행 상황을 설명.

4. UI/UX 디자인 업데이트 (김태영)
   - 김태영은 UI/UX 디자인 작업에 대한 업데이트를 공유.
   - 사용자 경험 향상과 인터페이스 디자인에 대한 계획을 설명.

5. 다음 주 목표 설정
   - 팀원들 간의 토론 및 협의 후, 다음 주까지의 목표를 설정.
   - 주요 개발 완료, UI/UX 디자인 최종화, 테스트 준비 등에 중점을 둠.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 황지연 담당.
   - 팀원들 간의 협력과 의사소통을 강조.

### Example Selector 의 유사도 검색 문제 해결
* 기존의 Example Selector는 유사도 계산 시 주로 instruction만을 사용합니다. 이로 인해 적절한 예제를 선택하지 못하는 문제가 발생할 수 있습니다. 
* instruction뿐만 아니라 input도 함께 사용하여 유사도를 계산합니다. 이 접근 방식을 통해 더 정확한 예제 선택이 가능